In [15]:
#%matplotlib inline
import os
import numpy as np
from PIL import Image
import matplotlib as mp
from matplotlib import pyplot,pylab
plt = pyplot
import scipy
from __future__ import division
import seaborn as sns
sns.set_style('white')
import string
import pandas as pd
from scipy import stats

import json
import pymongo as pm

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

## purpose

* upload sketches to S3 [maybe do this later]
* build stimulus dictionary and write to database

## upload sketches to S3 [todo later]

## build stimulus dictionary

In [31]:
## read in experimental metadata file (CSV)
path_to_metadata = 'sketchpad_basic_allcats.csv'
_meta = pd.read_csv(path_to_metadata)   ### raw meta with all categories

## subset by chairs
meta_chairs = _meta[_meta['category']=='chair'] ### subsetted meta with just chairs
meta_chairs.reset_index(inplace=True)

## assign which meta we will actually upload to mongo in this session
category_flag = 'chairs' ## options: ['allcats','chairs']

if category_flag == 'chairs':
    meta = meta_chairs
elif category_flag == 'allcats':
    meta = _meta

In [32]:
## add parts list
parts =[]
for i in range(meta.shape[0]-1):
    if meta.category[i]=="chair":
        parts.append(["backrest,armrest,seat,leg"])        
    if meta.category[i]=="dog":
        parts.append(["eye,mouth,ear,head,neck,body,leg,tail"])
    if meta.category[i] == "bird":
        parts.append(["beak,head,body,wings,feet,tail"])
    if meta.category[i] == "car":
        parts.append(["bumper,lights,windshield,body,window,door,wheel"])
meta = meta.assign(parts=pd.Series(parts))

In [34]:
## add iteration name information
_iterationName = 'sketchpad_basic_{}'.format(category_flag)
iterationName = [_iterationName]*len(meta)
meta = meta.assign(iterationName=pd.Series(iterationName))

In [35]:
## svg string formatting
svg = []
for i,d in meta.iterrows():    
    splitted = d['svg'].split("'") ## parse string to re-split up into strokes
    svgString = [i for i in splitted if i[0]=='M'] ## check to make sure it is a real start of a spline
    svg.append(svgString)
meta = meta.assign(svg=pd.Series(svg)) 

In [36]:
## add empty games list
games = [[] for i in np.arange(len(meta))]
meta = meta.assign(games=pd.Series(games))

In [37]:
## how many games worth of data do we have?
print '{} unique games worth of data.'.format(len(np.unique(meta.gameID.values)))
print '{} unique sketches.'.format(len(meta))

## write out metadata to json file

## for example:
stimdict = meta.to_dict(orient='records')
stimdict
import json
with open('annotation_meta.js', 'w') as fout:
     json.dump(stimdict, fout)

93 unique games worth of data.
744 unique sketches.


### upload stim dictionary to mongo (db = 'stimuli', collection='sketchpad_basic_recog')

In [8]:
## load in the JSON that contains the svgData, object labels, and part labels
J = json.loads(open('annotation_meta_chairs.js',mode='ru').read())
assert len(J)==len(meta)

In [9]:
# set vars 
auth = pd.read_csv('auth.txt', header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'rxdhawkins.me' ## cocolab ip address

# have to fix this to be able to analyze from local
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')

In [10]:
## define the dbname and collection name
db = conn['stimuli']
coll = db['svg_annotation_sketchpad_basic_chairs']

In [11]:
## actually add data now to the database (iff reallyRun==True)
reallyRun = True
if reallyRun:
    for (i,j) in enumerate(J):
        if i%250==0:
            print ('%d of %d' % (i,len(J)))
        coll.insert_one(j)
reallyRun = False        

0 of 744
250 of 744
500 of 744


In [12]:
## How many sketches do we have in the database?
print 'We have {} sketches.'.format(coll.count())

## What kind of sketches do we have in the database?
print 'We have these kinds: {}'.format(str(coll.distinct('category')))

We have 744 sketches.
We have these kinds: [u'chair']


In [13]:
meta

,X,gameID,trialNum,condition,target,drawDuration,outcome,response,numStrokes,Distractor1,...,Distractor3,pose,iteration,svg,viewerRT,mean_intensity,category,parts,iterationName,games
0,1,7004-6ff0964c-ff95-40f9-8696-d6a8931c94d5,1,further,knob,18.389,True,knob,7,crow,...,basset,35,pilot2,[],6.656,0.0347,chair,"[backrest,armrest,seat,leg]",sketchpad_basic_chairs,[]
1,2,7004-6ff0964c-ff95-40f9-8696-d6a8931c94d5,9,closer,waiting,21.246,True,waiting,11,leather,...,squat,35,pilot2,[],1.793,0.0722,chair,"[backrest,armrest,seat,leg]",sketchpad_basic_chairs,[]
2,3,7004-6ff0964c-ff95-40f9-8696-d6a8931c94d5,10,closer,inlay,25.374,True,inlay,14,leather,...,squat,35,pilot2,[],1.847,0.0566,chair,"[backrest,armrest,seat,leg]",sketchpad_basic_chairs,[]
3,4,7004-6ff0964c-ff95-40f9-8696-d6a8931c94d5,17,further,straight,4.712,True,straight,5,nightingale,...,goldenretriever,35,pilot2,[],31.134,0.0243,chair,"[backrest,armrest,seat,leg]",sketchpad_basic_chairs,[]
4,5,7004-6ff0964c-ff95-40f9-8696-d6a8931c94d5,23,closer,leather,37.167,True,leather,14,inlay,...,squat,35,pilot2,[],6.244,0.0663,chair,"[backrest,armrest,seat,leg]",sketchpad_basic_chairs,[]
5,6,7004-6ff0964c-ff95-40f9-8696-d6a8931c94d5,25,closer,squat,33.205,True,squat,16,leather,...,waiting,35,pilot2,[],1.505,0.0542,chair,"[backrest,armrest,seat,leg]",sketchpad_basic_chairs,[]
6,7,7004-6ff0964c-ff95-40f9-8696-d6a8931c94d5,26,further,woven,12.926,True,woven,8,tomtit,...,bloodhound,35,pilot2,[],4.021,0.0421,chair,"[backrest,armrest,seat,leg]",sketchpad_basic_chairs,[]
7,8,7004-6ff0964c-ff95-40f9-8696-d6a8931c94d5,29,further,sling,7.536,True,sling,6,robin,...,bullmastiff,35,pilot2,[],2.633,0.0394,chair,"[backrest,armrest,seat,leg]",sketchpad_basic_chairs,[]
8,9,6224-40ab6004-b7c1-4d1d-a308-a061099290fc,1,closer,straight,13.853,True,straight,7,squat,...,waiting,35,pilot2,[],8.662,0.0488,chair,"[backrest,armrest,seat,leg]",sketchpad_basic_chairs,[]
9,10,6224-40ab6004-b7c1-4d1d-a308-a061099290fc,4,further,knob,10.920,True,knob,6,pigeon,...,chihuahua,35,pilot2,[],3.621,0.0270,chair,"[backrest,armrest,seat,leg]",sketchpad_basic_chairs,[]
